<a href="https://colab.research.google.com/github//pylabel-project/samples/blob/main/pylabel2azure_custom_vision.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 
# Upload Annotations to Azure Custom Vision 
Custom Vision, part of the Azure Cognitive Services family, is a solution for training and deploying custom computer vision models. Custom Vision includes an API to upload images and annotations to train a custom model. Using PyLabel you can import existing labels in COCO, YOLOv5, or VOC format and then upload the dataset to Custom Vision.  

This notebook demonstrates how to import a custom dataset in YOLO format to Custom Vision. To complete the steps you will need an Azure Account and a Custom Vision subscription. Follow [this tutorial on the the Custom Vision site](https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/quickstarts/object-detection?tabs=visual-studio&pivots=programming-language-python) to setup your account and make sure it is working before using this notebook to import a custom dataset. When you are ready to use this notebook to upload a custom dataset, it is recommended to open https://www.customvision.ai/ so you can see the results of the commands you are performing through the API. 

In [ ]:
%pip install azure-cognitiveservices-vision-customvision
%pip install pylabel

#Import Azure cognitive services libraries 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#Import other libraries used in this notebook 
import os, zipfile
from pathlib import PurePath
from os.path import exists
from decimal import *

from pylabel import importer

In [ ]:
# Replace with your Azure endpoint and subscription keys.
ENDPOINT = ""
training_key = ""
prediction_key = ""
prediction_resource_id = ""

In [ ]:
#Initialize objects used by Azure Congitive vision
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)


In [ ]:
for domain in trainer.get_domains():
    # Print the domain name
    print(domain.name + " - " + domain.type)

In [ ]:
#Create a new project
#Change the domain name accordingly
publish_iteration_name = "detectModel"
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General (compact) [S1]")
project = trainer.create_project("Imported Weed Detection", domain_id=obj_detection_domain.id)
#If you browse to https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset"

## Download Custom Dataset 
For this demonstration we will download 100 images from the <a href="https://github.com/pylabel-project/datasets_models#squirrels-and-nuts">squirrels and nuts dataset with annotations in YOLOv5 format.</a> PyLabel can also import datasets in COCO and PASCAL VOC format. 

In [ ]:
%%capture
os.makedirs("data/", exist_ok=True)
!wget "https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_train.zip?raw=true" -O data/squirrelsandnuts_train.zip
with zipfile.ZipFile("data/squirrelsandnuts_train.zip", 'r') as zip_ref:
    zip_ref.extractall("data/")

In [ ]:
#Import annotations as a PyLabel dataset
dataset = importer.ImportYoloV5(path="C:/Users/myuser/train/labels",
        path_to_images="C:/Users/myuser/train/images", 
        img_ext="jpg",
        cat_names=['Lolium_Multiflorum','Conyza_Bonariensis', 'Digitaria_Insularis', 'Eleusine_Indica', 'Amaranthus_Palmeri']
    )
dataset.df.head(3)

## Import to Azure Custom Vision
PyLabel stores the annotations as a pandas dataframe. Now you can use extract the annotations from the dataframe and use it as inputs to the Custom Vision APIs. 

The first step is to create tags for each of the classes in your custom dataset. A list of class names is available in the dataset.analyze.classes property. 

In [ ]:
print(dataset.analyze.classes)
#Create a tag for each class and store then in a dict where the class name is the key
tags = {}
for class_name in dataset.analyze.classes:
    tag = trainer.create_tag(project.id, class_name)
    tags[class_name] = tag

Now if you check your account on https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset" with 2 tags added: Squirrels and Nuts. 

You are ready to upload your images and annotations. For each image in your dataset you will need to add "Regions" for each bounding box and then upload the image and annotations. 

In [ ]:
#Iterate the rows for each image in the dataframe
for img_filename, img_df in dataset.df.groupby('img_filename'):
    img_path = str(PurePath(dataset.path_to_annotations, str(img_df.iloc[0].img_folder), img_filename))
    assert exists(img_path), f"File does not exist: {img_path}"

    #Create a region object for each bounding box in the dataset 
    regions = []
    for index, row in img_df.iterrows():

        #Normalize the boundings box coordinates between 0 and 1
        x = Decimal(row.ann_bbox_xmin / row.img_width).min(1)
        y = Decimal(row.ann_bbox_ymin / row.img_height).min(1)
        w = Decimal(row.ann_bbox_width / row.img_width).min(1-x)
        h = Decimal(row.ann_bbox_height / row.img_height).min(1-y)
        
        regions.append(Region(
                tag_id=tags[row.cat_name].id, 
                left=x,
                top=y,
                width=w,
                height=h
            )
        )

    #Create an object with the image and all of the annotations for that image
    with open(img_path, mode="rb") as image_contents:
        image_and_annotations = [ImageFileCreateEntry(name=img_filename, contents=image_contents.read(), regions=regions)]

    #Upload the image and all annnotations for that image
    upload_result = trainer.create_images_from_files(
            project.id, 
            ImageFileCreateBatch(images=image_and_annotations)
        )
    
    #If upload is not successful, print details about that image for debugging 
    if not upload_result.is_batch_successful:
        print("Image upload failed.")
        for image in upload_result.images:
            print(img_path)
            print("Image status: ", image.status)
            print(regions)

#This will take a few minutes 
print("Upload complete")


Now you should see all of your images uploaded to https://www.customvision.ai/.
<p>
<img src="https://raw.githubusercontent.com/pylabel-project/datasets_models/main/pylabel_assets/custom_vision_project.png" width=400>
<p>
Click and image to see the bounding boxes.
<p>
<img src="https://raw.githubusercontent.com/pylabel-project/datasets_models/main/pylabel_assets/custom_vision_image.png" width=400>
<p>

Now you are ready to train a model, which you can do at https://www.customvision.ai/. 
- If find a problem with this notebook, please report it as an issue here: https://github.com/pylabel-project/pylabel/issues 
- If have other questions, please start a discussion here: https://github.com/pylabel-project/pylabel/discussions. 